In [1]:
import sys, os
from numpy import *
from matplotlib.pyplot import *
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 100

%load_ext autoreload
%autoreload 2

A baseline softmax classifier

Data preprocessing

In [2]:
import data_utils.utils as du
import data_utils.ner_base as nerbase

In [3]:
# # DONE: generating training set
# import json

# f = open('data/train','w')

# with open('data/buzs_labeled.json') as json_file:
#     for line in json_file:
#         review = json.loads(line)
#         for tuple in review["text"]:
#             tuple = tuple[0] + "\t" + tuple[1] + "\n"
#             data = tuple.encode("ascii","ignore")
#             if data.strip() == "-":
#                 data = "\n"
#             f.write(data)
            

In [4]:
# VOCAB_SIZE = 10000

# # TODO: add food word

# # generating vocab
# fv = open('data/vocab.txt', 'w')
# fv.write("UUUNKKK\n")
# fv.write("<s>\n")
# fv.write("</s>\n")

# cnt = 0
# with open('data/vocabulary.txt') as vocab_file:
#     for line in vocab_file:
#         if cnt < VOCAB_SIZE:
#             word = line.split(' ')[0] + '\n'
#             if word.strip() != '':
#                 fv.write(word.encode("ascii","ignore"))
#         cnt += 1


In [5]:
# stop word and UUUNKKK should include in the dictionary
word_to_num, num_to_word = nerbase.load_wv('data/vocab.txt')

tagnames = ["O", "B", "I", "L", "U"]
num_to_tag = dict(enumerate(tagnames))
tag_to_num = du.invert_dict(num_to_tag)

wsize = 3
pad = (wsize - 1)/2

X_train, y_train, words_train = du.generateData('data/train_small', word_to_num, tag_to_num, wsize)
# X_dev, y_dev, words_dev = du.generateData('data/dev_small', word_to_num, tag_to_num, wsize)


In [6]:


####! not same length ###
print len(X_train)
print len(words_train)


95230
95230


Feature extractor

In [7]:
# baseline features:

# context
# capitalize word
# previous tokens...
##### FEATURES TO ADD #####
def isCapital(x):
    if len(x) > 0:
        return x[0].isupper()
    return false

def extractFeatures(X, words, windowsize, word_to_num):
    fv1 = contextFeature(X, windowsize, word_to_num)
    fv2 = capitalFeature(words)
    fv = column_stack((fv1,fv2))
    return fv2

def contextFeature(X, windowsize, word_to_num):
    m = len(word_to_num)
    onehot_x = identity(m)

    wdim = m * windowsize
    N = X.shape[0]
    fvec = onehot_x[X].reshape(N, wdim)
    return fvec

def capitalFeature(words):
    fvec = array([isCapital(ww) for ww in words]).astype(int)
    return fvec

Training classifier

In [13]:
# X_train, y_train = du.docs_to_windows(docs_train, word_to_num, tag_to_num)
def trainSubset(size, X_train, y_train, words_train):
    X_train_s = X_train[:size]
    y_train_s = y_train[:size]
    words_train_s = words_train[:size]
    return X_train_s, y_train_s, words_train_s


# TRAIN_SIZE = 500
# X_s, y_s, w_s = trainSubset(TRAIN_SIZE, X_train, y_train, words_train)
X_s = X_train
y_s = y_train
w_s = words_train


In [ ]:
fv = extractFeatures(X_s, w_s, wsize, word_to_num)

In [11]:
### just trying###
X = X_train
m = len(word_to_num)
onehot_x = identity(m)

wdim = m * wsize
N = X.shape[0]
fvec = onehot_x[X].reshape(N, wdim)

In [14]:
print fv.shape

(95230,)


Training using sklearn logistic regression package

In [32]:
from sklearn import linear_model
lr = linear_model.LogisticRegression(solver='lbfgs', multi_class='multinomial')
lr.fit(fv, y_s)
y_pred = lr.predict(fv)
print nerbase.full_report(y_s, y_pred, tagnames)

             precision    recall  f1-score   support

          O       0.97      1.00      0.99       430
          B       1.00      0.79      0.88        19
          I       1.00      0.85      0.92        13
          L       1.00      0.84      0.91        19
          U       1.00      0.79      0.88        19

avg / total       0.97      0.97      0.97       500

None


Evaluation metric

In [33]:
print len(X_dev)
print len(X_train)

8486
9082


In [35]:
print fv.shape
print fvdev.shape

(500, 29989)
(8486, 29989)


Testing using dummy dataset

In [36]:
fvdev = extractFeatures(X_dev, words_dev, windowsize, word_to_num)
yp = lr.predict(fvdev)

nerbase.full_report(y_dev, yp, tagnames) # full report, helpful diagnostics

             precision    recall  f1-score   support

          O       0.88      1.00      0.93      7392
          B       0.25      0.00      0.00       403
          I       0.00      0.00      0.00       196
          L       0.00      0.00      0.00       403
          U       0.09      0.05      0.07        92

avg / total       0.78      0.87      0.81      8486

